In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 16.4 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
untrained_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
company database:
{}

### Input:
sql prompt:
{}

### Response:
sql:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    company_database = examples["sql_context"]
    prompts       = examples["sql_prompt"]
    sqls      = examples["sql"]
    # explanation = examples["sql_explanation"]
    texts=[]
    for company_database, prompts,sqls in zip(company_database, prompts,sqls):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(company_database, prompts,sqls) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset['text'][1]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\ncompany database:\nCREATE TABLE equipment_maintenance (equipment_type VARCHAR(255), maintenance_frequency INT);\n\n### Input:\nsql prompt:\nList all the unique equipment types and their corresponding total maintenance frequency from the equipment_maintenance table.\n\n### Response:\nsql:\nSELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;\n<|end_of_text|>'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,  # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use this for WandB etc
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.741 GB.
11.338 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,0.323400
2,0.308200
3,0.252700
4,0.189600
5,0.198000
6,0.173700
7,0.139100
8,0.202300
9,0.235700
10,0.212000


In [ ]:
alpaca_prompt_sql_only = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
company database: {}

### Input:
sql prompt: {}

### Response:
sql: {}

"""


from unsloth import FastLanguageModel

# Enable faster inference
FastLanguageModel.for_inference(model)

# 👇 Replace these with ANY new test case
company_database = """CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT);
INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South');
CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE);
INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'),
(2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');"""

sql_prompt = "What is the total volume of timber sold by each salesperson, sorted by salesperson?"


# Format the input with empty SQL and explanation slots for generation
input_prompt = alpaca_prompt.format(company_database, sql_prompt, "", "")

# Tokenize and move to CUDA
inputs = tokenizer([input_prompt], return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    use_cache=True,
    do_sample=False,
    temperature=0.7,
)

# Decode and print
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(generated_text)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
company database:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); 
INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); 
CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); 
INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), 
(2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

### Input:
sql prompt:
What is the total volume of timber sold by each salesperson, sorted by salesperson?

### Response:
sql:

SELECT s.name, SUM(ts.volume) as total_volume FROM salesperson s JOIN timber_sales ts ON s.salesperson_id = ts.salesperson_id GROUP BY s.name ORDER BY total_volume DESC;



In [ ]:
explanation_prompt = """Below is an instruction that describes a task. Given a SQL query and a company database schema, explain what the query is doing.

### Instruction:
company database:
{}

### SQL Query:
sql:{}

### Response:
explanation:{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    company_database = examples["sql_context"]
    sqls       = examples["sql"]
    explanation = examples["sql_explanation"]
    texts=[]
    for company_database, sqls,explanation in zip(company_database, sqls,explanation):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(company_database, sqls,explanation) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset1 = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
dataset1 = dataset.map(formatting_prompts_func, batched = True,)



Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset1['text'][1]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\ncompany database:\nCREATE TABLE equipment_maintenance (equipment_type VARCHAR(255), maintenance_frequency INT);\n\n### Input:\nsql prompt:\nSELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;\n\n### Response:\nsql:\nThis query groups the equipment_maintenance table by equipment_type and calculates the sum of maintenance_frequency for each group, then returns the equipment_type and the corresponding total_maintenance_frequency.\n<|end_of_text|>'

In [ ]:
untrained_model = FastLanguageModel.get_peft_model(
    untrained_model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = untrained_model,
    tokenizer = tokenizer,
    train_dataset = dataset1,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,  # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use this for WandB etc
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.781800
2,1.808000
3,1.695100
4,1.640800
5,1.415100
6,1.247000
7,1.164500
8,0.935900
9,0.851800
10,0.813000


In [ ]:
def generate_sql_and_explanation(company_database, sql_prompt):
    sql_input_prompt = alpaca_prompt.format(company_database, sql_prompt, "", "")
    sql_tokens = tokenizer([sql_input_prompt], return_tensors="pt").to("cuda")
    sql_output = model.generate(**sql_tokens, max_new_tokens=64)
    sql_text = tokenizer.batch_decode(sql_output, skip_special_tokens=True)[0]
    extracted_sql = extract_sql_from_response(sql_text)

    explanation_input = explanation_prompt.format(company_database, extracted_sql, "")
    exp_tokens = tokenizer([explanation_input], return_tensors="pt").to("cuda")
    explanation_output = untrained_model.generate(**exp_tokens, max_new_tokens=100)
    explanation_text = tokenizer.batch_decode(explanation_output, skip_special_tokens=True)[0]

    return sql_text, explanation_text


In [ ]:
FastLanguageModel.for_inference(untrained_model)

In [ ]:
company_database = """
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT);
INSERT INTO salesperson (salesperson_id, name, region) VALUES
(1, 'John Doe', 'North'), (2, 'Jane Smith', 'South');
CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE);
INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES
(1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');
"""

sql_prompt = "What is the total volume of timber sold by each salesperson, sorted by salesperson?"

# Call the function
sql_query, explanation = generate_sql_and_explanation(company_database, sql_prompt)

print("SQL Query:\n", sql_query)
print("\nExplanation:\n", explanation)


SQL Query:
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
company database:

CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); 
INSERT INTO salesperson (salesperson_id, name, region) VALUES 
(1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); 
CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); 
INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES 
(1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');


### Input:
sql prompt:
What is the total volume of timber sold by each salesperson, sorted by salesperson?

### Response:
sql:

SELECT s.name, SUM(t.volume) as total_volume FROM salesperson s JOIN timber_sales t ON s.salesperson_id = t.salesperson_id GROUP BY s.name ORDER BY total_volume DESC;


Explanation:
 Below is an instruction that describes

In [ ]:
import re

def extract_sql_from_response(response_text):
    match = re.search(r"sql:\s*(SELECT.*?);?$", response_text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # fallback if no "sql:" is found — just return the whole response
        return response_text.strip()


In [ ]:
from transformers import AutoTokenizer
from peft import PeftModel
import os

# Set your paths
sql_save_path = "/content/saved_models/sql_model"
exp_save_path = "/content/saved_models/explanation_model"

# Create folders if not exist
os.makedirs(sql_save_path, exist_ok=True)
os.makedirs(exp_save_path, exist_ok=True)

# ✅ Save SQL Model
# If you used PEFT (e.g. Unsloth), use .save_pretrained on PeftModel
model.save_pretrained(sql_save_path)
tokenizer.save_pretrained(sql_save_path)  # Assuming both models use the same tokenizer

# ✅ Save Explanation Model
untrained_model.save_pretrained(exp_save_path)
# No need to save tokenizer again if same

print("✅ Models saved successfully.")


✅ Models saved successfully.


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# Load tokenizer (from sql model)
tokenizer = AutoTokenizer.from_pretrained("saved_models/sql_model")

# Load base model (same as what you used for fine-tuning, e.g., Llama-3 8B, Mistral etc.)
base_model_id = "unsloth/llama-3-8b-bnb-4bit"  # or whatever base model you used
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)

# Load adapters
sql_model = PeftModel.from_pretrained(base_model, "saved_models/sql_model")
explanation_model = PeftModel.from_pretrained(base_model, "saved_models/explanation_model")

# Prompts
alpaca_prompt = """### Company Database:\n{0}\n### Question:\n{1}\n### SQL:\n{2}\n### Explanation:\n{3}"""
explanation_prompt = """### Company Database:\n{0}\n### SQL:\n{1}\n### Explanation:\n{2}"""

def extract_sql_from_response(text):
    # A basic way to extract SQL line (adjust if needed)
    for line in text.splitlines():
        if "SELECT" in line.upper():
            return line.strip()
    return text.strip()

def generate_sql_and_explanation(company_database, sql_prompt):
    # Step 1: Generate SQL
    sql_input_prompt = alpaca_prompt.format(company_database, sql_prompt, "", "")
    sql_tokens = tokenizer([sql_input_prompt], return_tensors="pt").to("cuda")
    sql_output = sql_model.generate(**sql_tokens, max_new_tokens=64)
    sql_text = tokenizer.batch_decode(sql_output, skip_special_tokens=True)[0]
    extracted_sql = extract_sql_from_response(sql_text)

    # Step 2: Generate Explanation
    explanation_input = explanation_prompt.format(company_database, extracted_sql, "")
    exp_tokens = tokenizer([explanation_input], return_tensors="pt").to("cuda")
    explanation_output = explanation_model.generate(**exp_tokens, max_new_tokens=100)
    explanation_text = tokenizer.batch_decode(explanation_output, skip_special_tokens=True)[0]

    return extracted_sql, explanation_text

# Gradio UI
iface = gr.Interface(
    fn=generate_sql_and_explanation,
    inputs=[
        gr.Textbox(label="Company Database", lines=10, placeholder="Paste SQL schema here..."),
        gr.Textbox(label="SQL Prompt", placeholder="e.g., What is the total volume sold by each salesperson?")
    ],
    outputs=[
        gr.Textbox(label="Generated SQL"),
        gr.Textbox(label="Explanation")
    ],
    title="SQL + Explanation Generator"
)

iface.launch()


In [ ]:
pip install transformers peft gradio accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 389.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 766.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 910.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit